In [39]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 약간의 전처리 과정

In [149]:
df = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/first_model_data.csv')
temp = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/youtube.csv')

In [150]:
# 시작과 끝을 분리해준다.
week = temp['주간'].str.split('-')
# 원래 데이터와 형태 맞춰주기 위해
# 그리고 유튜브 주기랑 가온차트 주기가 다르다. 
# 5를 빼주면 가온차트와 같아진다. 
start = []
end = []
for i in week:
    start.append(str(datetime.datetime.strptime(i[0],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])
    end.append(str(datetime.datetime.strptime(i[1],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])

In [151]:
# 전처리한 값 추가
temp['start'] = start
temp['end'] = end

In [152]:
# song과 날짜로 합칠거라서 song 전처리 해준다 
# 이유: 이름이 다른 사람들이 존재 ex) 조정석 == Jo Jung-suk
song = []
for i in temp['song']:
    a = re.split('[(-)]+' ,i)[0]
    song.append(a.strip())

In [153]:
temp['song'] = song
temp.head()

,주간,순위,song,name,lastweek_rank,지속기간,변경,조회수,start,end
0,20200417-20200423,1위,WANNABE,ITZY,지난주 #1,7 weeks,-9.5%,2.48M,2020-04-12,2020-04-18
1,20200417-20200423,2위,덤더럼,에이핑크,지난주 #4,2 weeks,30.1%,2.1M,2020-04-12,2020-04-18
2,20200417-20200423,3위,돌덩이,하현우,지난주 #2,9 weeks,-16.4%,1.66M,2020-04-12,2020-04-18
3,20200417-20200423,4위,시작,가호,지난주 #5,11 weeks,-10.6%,1.38M,2020-04-12,2020-04-18
4,20200417-20200423,5위,아로하,Jo Jung-suk,지난주 #11,4 weeks,26.7%,1.32M,2020-04-12,2020-04-18


In [154]:
# 필요없는 값 제거
df_for_merge = temp.drop(columns = ['주간','lastweek_rank','지속기간','변경','name'])

In [155]:
# 이름 변경
df_you = df_for_merge.rename(columns = {'start':'st_day', 'end':'ed_day','song':'name'})

In [156]:
df_you['st_day']

0        2020-04-12
1        2020-04-12
2        2020-04-12
3        2020-04-12
4        2020-04-12
            ...    
10395    2018-04-22
10396    2018-04-22
10397    2018-04-22
10398    2018-04-22
10399    2018-04-22
Name: st_day, Length: 10400, dtype: object

In [157]:
a = []
for i in df_you['st_day']:
    a.append(i.split('-'))

In [158]:
year = []
day = []
month = []
for i in range(0,len(a)):
    years = int(a[i][0][2:4])
    months = int(a[i][1])
    days = int(a[i][2])
    year.append(years)
    month.append(months)
    day.append(days)

In [159]:
df_you['year'] = year
df_you['month'] = month
df_you['day'] = day

In [160]:
df_you = df_you.drop(columns = ['st_day','ed_day'])

In [161]:
순위 = []
조회수 = []
# 조회수 전처리
for i in df_you['조회수']:
    if i == 0:
        조회수.append(0)
    elif i != 0:
        if 'M' in i:
            b = i.replace('M','00000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))
        elif 'K' in i:
            b = i.replace('K','0000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))

# 순위 전처리
for i in df_you['순위']:
    if i == 0:
        순위.append(i)
    elif i != 0:
        i = i.replace('위','')
        순위.append(int(i))

In [162]:
df_you['순위'] = 순위
df_you['조회수'] = 조회수

In [163]:
df_you

,순위,name,조회수,year,month,day
0,1,WANNABE,24800000,20,4,12
1,2,덤더럼,2100000,20,4,12
2,3,돌덩이,16600000,20,4,12
3,4,시작,13800000,20,4,12
4,5,아로하,13200000,20,4,12
...,...,...,...,...,...,...
10395,96,야생화,1660000,18,4,22
10396,97,북,1650000,18,4,22
10397,98,DINOSAUR,1630000,18,4,22
10398,99,"Not Spring, Love, or Cherry Blossoms",1620000,18,4,22


In [164]:
# 유튜브와 합치기
a = pd.merge(df,df_you,on=['name','year','month','day'],how = 'left')

In [165]:
a

,Unnamed: 0,artist,name,previous_song_rank,st_day,X1,id,production,distributor,ed_day,...,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수
0,1,(여자)아이들,LATATA,NaN,2018.5.6,235,235,큐브엔터테인먼트,카카오 M,2018.5.12,...,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0
1,2,(여자)아이들,LATATA,NaN,2018.5.13,319,319,큐브엔터테인먼트,카카오 M,2018.5.19,...,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN
2,3,(여자)아이들,LATATA,NaN,2018.5.20,416,416,큐브엔터테인먼트,카카오 M,2018.5.26,...,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN
3,4,(여자)아이들,LATATA,NaN,2018.5.27,512,512,큐브엔터테인먼트,카카오 M,2018.6.2,...,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN
4,5,(여자)아이들,LATATA,NaN,2018.6.3,614,614,큐브엔터테인먼트,카카오 M,2018.6.9,...,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,17396,히키,사랑에 연습이 있었다면,NaN,2019.6.23,8701,8701,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,...,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN
17479,17397,히키,사랑에 연습이 있었다면,NaN,2019.6.30,8904,8904,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,...,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN
17480,17398,히키,사랑에 연습이 있었다면,NaN,2019.7.7,9141,9141,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,...,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN
17481,17399,히키,사랑에 연습이 있었다면,NaN,2019.7.14,9364,9364,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,...,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN


In [166]:
df = a.drop_duplicates(['artist','name','st_day','ed_day'],keep='last')

In [167]:
a = []
for i in list(df['장르']):
    b = i.split(' / ')
    a.append(b)
    
genre = []
for i in range(0,len(list(a))):
    genre.append(a[i][1])

df['genre'] = genre

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [168]:
df

,Unnamed: 0,artist,name,previous_song_rank,st_day,X1,id,production,distributor,ed_day,...,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수
0,1,(여자)아이들,LATATA,NaN,2018.5.6,235,235,큐브엔터테인먼트,카카오 M,2018.5.12,...,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0
1,2,(여자)아이들,LATATA,NaN,2018.5.13,319,319,큐브엔터테인먼트,카카오 M,2018.5.19,...,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN
2,3,(여자)아이들,LATATA,NaN,2018.5.20,416,416,큐브엔터테인먼트,카카오 M,2018.5.26,...,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN
3,4,(여자)아이들,LATATA,NaN,2018.5.27,512,512,큐브엔터테인먼트,카카오 M,2018.6.2,...,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN
4,5,(여자)아이들,LATATA,NaN,2018.6.3,614,614,큐브엔터테인먼트,카카오 M,2018.6.9,...,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,17396,히키,사랑에 연습이 있었다면,NaN,2019.6.23,8701,8701,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,...,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN
17479,17397,히키,사랑에 연습이 있었다면,NaN,2019.6.30,8904,8904,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,...,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN
17480,17398,히키,사랑에 연습이 있었다면,NaN,2019.7.7,9141,9141,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,...,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN
17481,17399,히키,사랑에 연습이 있었다면,NaN,2019.7.14,9364,9364,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,...,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN


In [171]:
df.to_csv('basic.csv')

In [172]:
df = df.drop(columns = ['X1','id','Unnamed: 0','name_artist','song_id','st_day','production','distributor','ed_day','장르','런타임','성별','score','rank','활동유형','wks','week','active_g','top_freq_g'])

In [173]:
df = df.drop(columns = ['dc_t_number','dc_t_recommend','dc_t_views','dc_m_recommend','runtime_g','dc_m_views','active'])

In [174]:
pd.set_option('display.max_columns', 999)

In [175]:
df.loc[df['previous_song_rank'].isnull(),'previous_song_rank'] = 200

In [176]:
df.loc[df['순위'].isnull(),'순위'] = 100
df.loc[df['조회수'].isnull(),'조회수'] = 0

In [177]:
df.loc[df['ndc_t_numb'].isnull(),'ndc_t_numb'] = 0
df.loc[df['ndc_t_rec'].isnull(),'ndc_t_rec'] = 0
df.loc[df['ndc_t_view'].isnull(),'ndc_t_view'] = 0

In [ ]:
df.to_csv('first_model_glm.csv')

위에는 전처리 과정. 다른 사람들과 협업함.

In [295]:
temp = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/production_score.csv')

In [333]:
temp

,artist,name,previous_song_rank,st_day,production,distributor,ed_day,score,rank,name_artist,장르,런타임,성별,활동유형,song_id,rank_g,year,month,day,wks,week,sex,song_type,genre,runtime,runtime_g,active_type,active,active_g,top_freq,top_freq_g,dc_t_number,dc_t_recommend,dc_t_views,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수,pd_score
0,(여자)아이들,LATATA,NaN,2018-05-06,큐브엔터테인먼트,카카오 M,2018.5.12,12149754,35,"(여자)아이들,LATATA",가요 / 댄스,03:23:00,여성,그룹,109,4,18,5,6,18,18_18,female,K_POP,댄스,203,Middle,Group,Group,Non_solo,24,High,0,0,0,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0,5.6
1,(여자)아이들,LATATA,NaN,2018-05-13,큐브엔터테인먼트,카카오 M,2018.5.19,19604420,19,"(여자)아이들,LATATA",가요 / 댄스,03:23:00,여성,그룹,109,2,18,5,13,19,18_19,female,K_POP,댄스,203,Middle,Group,Group,Non_solo,24,High,0,0,0,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN,3.9
2,(여자)아이들,LATATA,NaN,2018-05-20,큐브엔터테인먼트,카카오 M,2018.5.26,19342281,16,"(여자)아이들,LATATA",가요 / 댄스,03:23:00,여성,그룹,109,2,18,5,20,20,18_20,female,K_POP,댄스,203,Middle,Group,Group,Non_solo,24,High,0,0,0,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN,3.8
3,(여자)아이들,LATATA,NaN,2018-05-27,큐브엔터테인먼트,카카오 M,2018.6.2,21249155,12,"(여자)아이들,LATATA",가요 / 댄스,03:23:00,여성,그룹,109,2,18,5,27,21,18_21,female,K_POP,댄스,203,Middle,Group,Group,Non_solo,24,High,0,0,0,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN,3.4
4,(여자)아이들,LATATA,NaN,2018-06-03,큐브엔터테인먼트,카카오 M,2018.6.9,19646611,14,"(여자)아이들,LATATA",가요 / 댄스,03:23:00,여성,그룹,109,2,18,6,3,22,18_22,female,K_POP,댄스,203,Middle,Group,Group,Non_solo,24,High,0,0,0,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,히키,사랑에 연습이 있었다면,NaN,2019-06-23,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,6356941,101,"히키,사랑에 연습이 있었다면",가요 / 인디,04:14:00,여성,솔로,748,10,19,6,23,25,19_25,female,K_POP,인디,254,Long,Solo,Solo,Solo,8,Middle,0,0,0,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN,0.0
17359,히키,사랑에 연습이 있었다면,NaN,2019-06-30,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,6905082,104,"히키,사랑에 연습이 있었다면",가요 / 인디,04:14:00,여성,솔로,748,10,19,6,30,26,19_26,female,K_POP,인디,254,Long,Solo,Solo,Solo,8,Middle,0,0,0,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN,0.0
17360,히키,사랑에 연습이 있었다면,NaN,2019-07-07,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,4389674,141,"히키,사랑에 연습이 있었다면",가요 / 인디,04:14:00,여성,솔로,748,10,19,7,7,27,19_27,female,K_POP,인디,254,Long,Solo,Solo,Solo,8,Middle,0,0,0,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN,0.0
17361,히키,사랑에 연습이 있었다면,NaN,2019-07-14,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,3547026,164,"히키,사랑에 연습이 있었다면",가요 / 인디,04:14:00,여성,솔로,748,10,19,7,14,28,19_28,female,K_POP,인디,254,Long,Solo,Solo,Solo,8,Middle,0,0,0,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN,0.0


In [378]:
df = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/new_model.csv',encoding = 'utf8')

In [379]:
df = df.drop_duplicates(['artist','name','sex','ymd'],keep='last')

In [380]:
df = df.reset_index()

In [381]:
del df['index']

In [382]:
df['pd_score'] = temp['pd_score']

In [383]:
df = df.drop(columns = ['artist','name','year' ,'month' ,'day'])

In [384]:
del df['X1']

In [385]:
del df['X1_1']

In [386]:
del df['you_rank']

In [387]:
del df['previous_song_rank_g']

In [388]:
del df['ymd']

In [389]:
df = df.rename(columns = {'조회수':'total_view'})

In [390]:
df.loc[df['previous_ranking'].isnull(),'previous_ranking'] = '10'

In [391]:
df.to_csv('new_model_pred.csv')

In [392]:
df = pd.get_dummies(df, columns = ['sex'])
df = pd.get_dummies(df, columns = ['song_type'])
df = pd.get_dummies(df, columns = ['you_rank_g'])

In [417]:
df

,rank_g,runtime,top_freq,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,total_view,season_genre_score,pd_score,sex_female,sex_male,sex_mixed,song_type_K_POP,song_type_OST,song_type_Others,song_type_POP,you_rank_g_1,you_rank_g_2,you_rank_g_3,you_rank_g_4,you_rank_g_5,you_rank_g_6,you_rank_g_7,you_rank_g_8,you_rank_g_9,you_rank_g_10,season_fall,season_spring,season_summer,season_winter,genre_R&B/소울,genre_댄스,genre_드라마,genre_락,genre_랩/힙합,genre_발라드,genre_블루스/포크,genre_애니메이션/게임,genre_인디,genre_일렉트로니카,genre_전체,genre_정통,genre_캐롤,genre_트로트,genre_팝,genre_해외영화,active_type_Band,active_type_Duet,active_type_Group,active_type_Project,active_type_Solo,previous_ranking_1.0,previous_ranking_2.0,previous_ranking_3.0,previous_ranking_4.0,previous_ranking_5.0,previous_ranking_6.0,previous_ranking_7.0,previous_ranking_8.0,previous_ranking_9.0,previous_ranking_10.0,previous_ranking_10
0,4,0.523669,0.221154,0.80,1.000,0.0,0.0,0.0,0.086778,-0.093812,5.6,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2,0.523669,0.221154,0.60,0.303,0.0,0.0,0.0,0.000000,-0.093812,3.9,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,10,0.523669,0.221154,0.25,0.073,0.0,0.0,0.0,0.000000,0.111253,3.8,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,2,0.523669,0.221154,0.20,0.092,0.0,0.0,0.0,0.000000,-0.093812,3.4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,8,0.523669,0.221154,0.46,0.087,0.0,0.0,0.0,0.044578,-0.020121,3.2,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,10,0.674556,0.067308,0.31,0.251,0.0,0.0,0.0,0.000000,-0.021089,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
17359,9,0.674556,0.067308,0.41,1.000,0.0,0.0,0.0,0.000000,-0.021089,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
17360,10,0.674556,0.067308,0.41,0.460,0.0,0.0,0.0,0.000000,-0.021089,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
17361,10,0.674556,0.067308,0.22,0.165,0.0,0.0,0.0,0.000000,-0.021089,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0


In [393]:
df = pd.get_dummies(df, columns = ['season'])

In [394]:
df = pd.get_dummies(df, columns = ['genre'])

In [395]:
df = pd.get_dummies(df, columns = ['active_type'])

In [396]:
df = pd.get_dummies(df, columns = ['previous_ranking'])

In [397]:
from sklearn.preprocessing import MinMaxScaler

In [398]:
scaler = MinMaxScaler()

In [399]:
scaler.fit(df[['runtime','top_freq','gg_score','nv_score','ndc_t_numb','ndc_t_rec','ndc_t_view','total_view']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [400]:
df[['runtime','top_freq','gg_score','nv_score','ndc_t_numb','ndc_t_rec','ndc_t_view','total_view']] = scaler.transform(df[['runtime','top_freq','gg_score','nv_score','ndc_t_numb','ndc_t_rec','ndc_t_view','total_view']])

In [402]:
df['total_view'] = np.log(df['total_view']+1)

In [361]:
from sklearn.model_selection import train_test_split

In [373]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['rank_g']),df['rank_g'], test_size = 0.3)

# 사용 예정 모델
- XGboost
- LightGBM
- RandomForest
- Catboost
- Logistic regression

In [206]:
from sklearn.linear_model import LogisticRegression

In [203]:
import catboost

# Logistic Regression

In [584]:
model = LogisticRegression(random_state=1)

In [585]:
model.fit(X_train,y_train)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [586]:
pred_y = model.predict(X_test)

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [603]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.474


# RandomForest

In [363]:
from sklearn.ensemble import RandomForestClassifier

In [412]:
model = RandomForestClassifier(random_state =1, n_estimators = 500)

In [407]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# K-Fold Validation 
Classification일때는 Stratified K-Fold를 해줘야한다. 

In [408]:
stf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)

In [409]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [410]:
score

array([0.68140868, 0.69647252, 0.68390805, 0.67763158, 0.68914474,
       0.70016474, 0.69192751, 0.67821782, 0.70049505, 0.68811881])

In [411]:
score.mean()

0.6887489501802884

In [403]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [404]:
pred_y = model.predict(X_test)

In [405]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.681


# Catboost

In [204]:
from catboost import CatBoostClassifier

In [419]:
model = CatBoostClassifier()

In [420]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

Learning rate set to 0.089392
0:	learn: 1.9704652	total: 42.2ms	remaining: 42.1s
1:	learn: 1.8345892	total: 98ms	remaining: 48.9s
2:	learn: 1.7052067	total: 134ms	remaining: 44.6s
3:	learn: 1.6255881	total: 180ms	remaining: 44.8s
4:	learn: 1.5541384	total: 230ms	remaining: 45.8s
5:	learn: 1.4926899	total: 263ms	remaining: 43.6s
6:	learn: 1.4332859	total: 290ms	remaining: 41.1s
7:	learn: 1.3805460	total: 320ms	remaining: 39.7s
8:	learn: 1.3185854	total: 348ms	remaining: 38.4s
9:	learn: 1.2637379	total: 369ms	remaining: 36.5s
10:	learn: 1.2287827	total: 395ms	remaining: 35.5s
11:	learn: 1.1992983	total: 417ms	remaining: 34.3s
12:	learn: 1.1631857	total: 442ms	remaining: 33.5s
13:	learn: 1.1319458	total: 469ms	remaining: 33s
14:	learn: 1.0985694	total: 498ms	remaining: 32.7s
15:	learn: 1.0804842	total: 524ms	remaining: 32.2s
16:	learn: 1.0538729	total: 549ms	remaining: 31.7s
17:	learn: 1.0355641	total: 574ms	remaining: 31.3s
18:	learn: 1.0187404	total: 598ms	remaining: 30.9s
19:	learn: 1.

168:	learn: 0.6750945	total: 4.27s	remaining: 21s
169:	learn: 0.6742426	total: 4.3s	remaining: 21s
170:	learn: 0.6736087	total: 4.33s	remaining: 21s
171:	learn: 0.6722678	total: 4.36s	remaining: 21s
172:	learn: 0.6715335	total: 4.38s	remaining: 21s
173:	learn: 0.6706016	total: 4.41s	remaining: 20.9s
174:	learn: 0.6695496	total: 4.44s	remaining: 20.9s
175:	learn: 0.6686958	total: 4.46s	remaining: 20.9s
176:	learn: 0.6678187	total: 4.5s	remaining: 20.9s
177:	learn: 0.6668479	total: 4.53s	remaining: 20.9s
178:	learn: 0.6663337	total: 4.56s	remaining: 20.9s
179:	learn: 0.6656617	total: 4.58s	remaining: 20.9s
180:	learn: 0.6646474	total: 4.61s	remaining: 20.8s
181:	learn: 0.6634002	total: 4.63s	remaining: 20.8s
182:	learn: 0.6626734	total: 4.66s	remaining: 20.8s
183:	learn: 0.6615118	total: 4.69s	remaining: 20.8s
184:	learn: 0.6611408	total: 4.72s	remaining: 20.8s
185:	learn: 0.6602999	total: 4.75s	remaining: 20.8s
186:	learn: 0.6594001	total: 4.77s	remaining: 20.7s
187:	learn: 0.6584955	to

328:	learn: 0.5775614	total: 8.3s	remaining: 16.9s
329:	learn: 0.5768384	total: 8.33s	remaining: 16.9s
330:	learn: 0.5764827	total: 8.36s	remaining: 16.9s
331:	learn: 0.5759502	total: 8.38s	remaining: 16.9s
332:	learn: 0.5753950	total: 8.4s	remaining: 16.8s
333:	learn: 0.5748157	total: 8.43s	remaining: 16.8s
334:	learn: 0.5741964	total: 8.46s	remaining: 16.8s
335:	learn: 0.5736368	total: 8.48s	remaining: 16.8s
336:	learn: 0.5731202	total: 8.51s	remaining: 16.7s
337:	learn: 0.5722794	total: 8.54s	remaining: 16.7s
338:	learn: 0.5714890	total: 8.56s	remaining: 16.7s
339:	learn: 0.5708509	total: 8.58s	remaining: 16.7s
340:	learn: 0.5704349	total: 8.6s	remaining: 16.6s
341:	learn: 0.5698732	total: 8.63s	remaining: 16.6s
342:	learn: 0.5692186	total: 8.64s	remaining: 16.6s
343:	learn: 0.5687047	total: 8.66s	remaining: 16.5s
344:	learn: 0.5684286	total: 8.68s	remaining: 16.5s
345:	learn: 0.5679188	total: 8.7s	remaining: 16.5s
346:	learn: 0.5672797	total: 8.73s	remaining: 16.4s
347:	learn: 0.56

488:	learn: 0.5047637	total: 11.9s	remaining: 12.4s
489:	learn: 0.5044985	total: 11.9s	remaining: 12.4s
490:	learn: 0.5040302	total: 11.9s	remaining: 12.4s
491:	learn: 0.5035514	total: 12s	remaining: 12.4s
492:	learn: 0.5032998	total: 12s	remaining: 12.3s
493:	learn: 0.5030602	total: 12s	remaining: 12.3s
494:	learn: 0.5026275	total: 12s	remaining: 12.3s
495:	learn: 0.5019060	total: 12s	remaining: 12.2s
496:	learn: 0.5016295	total: 12.1s	remaining: 12.2s
497:	learn: 0.5013926	total: 12.1s	remaining: 12.2s
498:	learn: 0.5011839	total: 12.1s	remaining: 12.2s
499:	learn: 0.5008734	total: 12.1s	remaining: 12.1s
500:	learn: 0.5003135	total: 12.2s	remaining: 12.1s
501:	learn: 0.5000130	total: 12.2s	remaining: 12.1s
502:	learn: 0.4995969	total: 12.2s	remaining: 12.1s
503:	learn: 0.4993059	total: 12.2s	remaining: 12s
504:	learn: 0.4989498	total: 12.2s	remaining: 12s
505:	learn: 0.4987363	total: 12.3s	remaining: 12s
506:	learn: 0.4983753	total: 12.3s	remaining: 12s
507:	learn: 0.4979451	total: 1

647:	learn: 0.4468239	total: 15.7s	remaining: 8.52s
648:	learn: 0.4465486	total: 15.7s	remaining: 8.5s
649:	learn: 0.4463498	total: 15.7s	remaining: 8.48s
650:	learn: 0.4461938	total: 15.8s	remaining: 8.45s
651:	learn: 0.4455576	total: 15.8s	remaining: 8.43s
652:	learn: 0.4449428	total: 15.8s	remaining: 8.4s
653:	learn: 0.4446663	total: 15.8s	remaining: 8.38s
654:	learn: 0.4443995	total: 15.9s	remaining: 8.35s
655:	learn: 0.4441765	total: 15.9s	remaining: 8.32s
656:	learn: 0.4440210	total: 15.9s	remaining: 8.3s
657:	learn: 0.4438616	total: 15.9s	remaining: 8.28s
658:	learn: 0.4435311	total: 16s	remaining: 8.25s
659:	learn: 0.4432803	total: 16s	remaining: 8.23s
660:	learn: 0.4429233	total: 16s	remaining: 8.21s
661:	learn: 0.4425999	total: 16s	remaining: 8.18s
662:	learn: 0.4423985	total: 16s	remaining: 8.16s
663:	learn: 0.4422281	total: 16.1s	remaining: 8.13s
664:	learn: 0.4418487	total: 16.1s	remaining: 8.11s
665:	learn: 0.4417409	total: 16.1s	remaining: 8.09s
666:	learn: 0.4415350	tot

...

324:	learn: 0.5839630	total: 6.54s	remaining: 13.6s
325:	learn: 0.5832459	total: 6.56s	remaining: 13.6s
326:	learn: 0.5827663	total: 6.58s	remaining: 13.5s
327:	learn: 0.5822077	total: 6.6s	remaining: 13.5s
328:	learn: 0.5813682	total: 6.62s	remaining: 13.5s
329:	learn: 0.5810082	total: 6.64s	remaining: 13.5s
330:	learn: 0.5808616	total: 6.66s	remaining: 13.5s
331:	learn: 0.5803060	total: 6.68s	remaining: 13.4s
332:	learn: 0.5799139	total: 6.69s	remaining: 13.4s
333:	learn: 0.5794682	total: 6.71s	remaining: 13.4s
334:	learn: 0.5791475	total: 6.73s	remaining: 13.4s
335:	learn: 0.5784018	total: 6.76s	remaining: 13.4s
336:	learn: 0.5780949	total: 6.78s	remaining: 13.3s
337:	learn: 0.5777010	total: 6.8s	remaining: 13.3s
338:	learn: 0.5771857	total: 6.82s	remaining: 13.3s
339:	learn: 0.5767248	total: 6.84s	remaining: 13.3s
340:	learn: 0.5762983	total: 6.86s	remaining: 13.3s
341:	learn: 0.5758188	total: 6.88s	remaining: 13.2s
342:	learn: 0.5749191	total: 6.9s	remaining: 13.2s
343:	learn: 0.5

488:	learn: 0.5098360	total: 9.9s	remaining: 10.3s
489:	learn: 0.5095181	total: 9.92s	remaining: 10.3s
490:	learn: 0.5091407	total: 9.95s	remaining: 10.3s
491:	learn: 0.5088853	total: 9.97s	remaining: 10.3s
492:	learn: 0.5082908	total: 9.99s	remaining: 10.3s
493:	learn: 0.5078848	total: 10s	remaining: 10.3s
494:	learn: 0.5076843	total: 10s	remaining: 10.2s
495:	learn: 0.5069504	total: 10s	remaining: 10.2s
496:	learn: 0.5065667	total: 10.1s	remaining: 10.2s
497:	learn: 0.5060767	total: 10.1s	remaining: 10.2s
498:	learn: 0.5056811	total: 10.1s	remaining: 10.2s
499:	learn: 0.5051923	total: 10.1s	remaining: 10.1s
500:	learn: 0.5047153	total: 10.2s	remaining: 10.1s
501:	learn: 0.5040152	total: 10.2s	remaining: 10.1s
502:	learn: 0.5037701	total: 10.2s	remaining: 10.1s
503:	learn: 0.5033661	total: 10.2s	remaining: 10.1s
504:	learn: 0.5025672	total: 10.2s	remaining: 10s
505:	learn: 0.5018008	total: 10.3s	remaining: 10s
506:	learn: 0.5013918	total: 10.3s	remaining: 9.99s
507:	learn: 0.5009264	t

653:	learn: 0.4473734	total: 13.3s	remaining: 7.02s
654:	learn: 0.4471835	total: 13.3s	remaining: 7.01s
655:	learn: 0.4469042	total: 13.3s	remaining: 6.99s
656:	learn: 0.4465822	total: 13.3s	remaining: 6.97s
657:	learn: 0.4458973	total: 13.4s	remaining: 6.95s
658:	learn: 0.4456021	total: 13.4s	remaining: 6.92s
659:	learn: 0.4452017	total: 13.4s	remaining: 6.9s
660:	learn: 0.4447809	total: 13.4s	remaining: 6.88s
661:	learn: 0.4444741	total: 13.4s	remaining: 6.86s
662:	learn: 0.4440825	total: 13.5s	remaining: 6.84s
663:	learn: 0.4434312	total: 13.5s	remaining: 6.82s
664:	learn: 0.4430486	total: 13.5s	remaining: 6.79s
665:	learn: 0.4428838	total: 13.5s	remaining: 6.78s
666:	learn: 0.4426256	total: 13.5s	remaining: 6.75s
667:	learn: 0.4422950	total: 13.5s	remaining: 6.73s
668:	learn: 0.4418912	total: 13.6s	remaining: 6.71s
669:	learn: 0.4414209	total: 13.6s	remaining: 6.69s
670:	learn: 0.4409421	total: 13.6s	remaining: 6.67s
671:	learn: 0.4407748	total: 13.6s	remaining: 6.65s
672:	learn: 0

820:	learn: 0.3975820	total: 16.6s	remaining: 3.63s
821:	learn: 0.3973644	total: 16.6s	remaining: 3.6s
822:	learn: 0.3970937	total: 16.7s	remaining: 3.58s
823:	learn: 0.3967822	total: 16.7s	remaining: 3.56s
824:	learn: 0.3966406	total: 16.7s	remaining: 3.54s
825:	learn: 0.3962842	total: 16.7s	remaining: 3.52s
826:	learn: 0.3960771	total: 16.7s	remaining: 3.5s
827:	learn: 0.3957952	total: 16.8s	remaining: 3.48s
828:	learn: 0.3955522	total: 16.8s	remaining: 3.46s
829:	learn: 0.3953887	total: 16.8s	remaining: 3.44s
830:	learn: 0.3951448	total: 16.8s	remaining: 3.42s
831:	learn: 0.3948240	total: 16.8s	remaining: 3.4s
832:	learn: 0.3947234	total: 16.9s	remaining: 3.38s
833:	learn: 0.3944770	total: 16.9s	remaining: 3.36s
834:	learn: 0.3942870	total: 16.9s	remaining: 3.34s
835:	learn: 0.3937916	total: 16.9s	remaining: 3.32s
836:	learn: 0.3934920	total: 16.9s	remaining: 3.3s
837:	learn: 0.3933303	total: 17s	remaining: 3.28s
838:	learn: 0.3931410	total: 17s	remaining: 3.26s
839:	learn: 0.392956

980:	learn: 0.3542431	total: 19.8s	remaining: 383ms
981:	learn: 0.3540895	total: 19.8s	remaining: 363ms
982:	learn: 0.3539628	total: 19.8s	remaining: 343ms
983:	learn: 0.3536924	total: 19.8s	remaining: 323ms
984:	learn: 0.3533720	total: 19.9s	remaining: 303ms
985:	learn: 0.3532378	total: 19.9s	remaining: 282ms
986:	learn: 0.3530866	total: 19.9s	remaining: 262ms
987:	learn: 0.3529100	total: 19.9s	remaining: 242ms
988:	learn: 0.3527560	total: 20s	remaining: 222ms
989:	learn: 0.3526555	total: 20s	remaining: 202ms
990:	learn: 0.3522813	total: 20s	remaining: 182ms
991:	learn: 0.3521854	total: 20s	remaining: 161ms
992:	learn: 0.3520053	total: 20s	remaining: 141ms
993:	learn: 0.3517551	total: 20.1s	remaining: 121ms
994:	learn: 0.3514508	total: 20.1s	remaining: 101ms
995:	learn: 0.3511822	total: 20.1s	remaining: 80.7ms
996:	learn: 0.3509825	total: 20.1s	remaining: 60.6ms
997:	learn: 0.3508061	total: 20.1s	remaining: 40.4ms
998:	learn: 0.3505499	total: 20.2s	remaining: 20.2ms
999:	learn: 0.3503

In [422]:
score

array([0.6961507 , 0.70959803, 0.69129721, 0.69983553, 0.70888158,
       0.71252059, 0.68698517, 0.68646865, 0.70462046, 0.69719472])

In [423]:
model.fit(X_train,y_train)

Learning rate set to 0.089876
0:	learn: 1.9503996	total: 31.1ms	remaining: 31.1s
1:	learn: 1.7917291	total: 72.3ms	remaining: 36.1s
2:	learn: 1.6531455	total: 112ms	remaining: 37.2s
3:	learn: 1.5783543	total: 147ms	remaining: 36.5s
4:	learn: 1.5085315	total: 174ms	remaining: 34.6s
5:	learn: 1.4510193	total: 196ms	remaining: 32.4s
6:	learn: 1.4000396	total: 223ms	remaining: 31.6s
7:	learn: 1.3527139	total: 248ms	remaining: 30.7s
8:	learn: 1.2946166	total: 285ms	remaining: 31.4s
9:	learn: 1.2461618	total: 312ms	remaining: 30.9s
10:	learn: 1.2108755	total: 341ms	remaining: 30.7s
11:	learn: 1.1745367	total: 369ms	remaining: 30.4s
12:	learn: 1.1434420	total: 389ms	remaining: 29.5s
13:	learn: 1.1122556	total: 408ms	remaining: 28.7s
14:	learn: 1.0822737	total: 431ms	remaining: 28.3s
15:	learn: 1.0654654	total: 457ms	remaining: 28.1s
16:	learn: 1.0459336	total: 482ms	remaining: 27.9s
17:	learn: 1.0255711	total: 506ms	remaining: 27.6s
18:	learn: 1.0066034	total: 529ms	remaining: 27.3s
19:	learn

162:	learn: 0.6883304	total: 4.25s	remaining: 21.8s
163:	learn: 0.6876365	total: 4.27s	remaining: 21.8s
164:	learn: 0.6867167	total: 4.29s	remaining: 21.7s
165:	learn: 0.6863702	total: 4.32s	remaining: 21.7s
166:	learn: 0.6854172	total: 4.34s	remaining: 21.6s
167:	learn: 0.6848531	total: 4.36s	remaining: 21.6s
168:	learn: 0.6841480	total: 4.38s	remaining: 21.5s
169:	learn: 0.6835654	total: 4.39s	remaining: 21.5s
170:	learn: 0.6826159	total: 4.42s	remaining: 21.4s
171:	learn: 0.6818068	total: 4.43s	remaining: 21.3s
172:	learn: 0.6810543	total: 4.46s	remaining: 21.3s
173:	learn: 0.6802366	total: 4.48s	remaining: 21.3s
174:	learn: 0.6795414	total: 4.5s	remaining: 21.2s
175:	learn: 0.6787786	total: 4.53s	remaining: 21.2s
176:	learn: 0.6781904	total: 4.54s	remaining: 21.1s
177:	learn: 0.6775865	total: 4.56s	remaining: 21.1s
178:	learn: 0.6770571	total: 4.58s	remaining: 21s
179:	learn: 0.6763215	total: 4.6s	remaining: 21s
180:	learn: 0.6758451	total: 4.62s	remaining: 20.9s
181:	learn: 0.6752

325:	learn: 0.5912836	total: 7.61s	remaining: 15.7s
326:	learn: 0.5907736	total: 7.63s	remaining: 15.7s
327:	learn: 0.5901781	total: 7.65s	remaining: 15.7s
328:	learn: 0.5898415	total: 7.67s	remaining: 15.6s
329:	learn: 0.5894237	total: 7.69s	remaining: 15.6s
330:	learn: 0.5891023	total: 7.71s	remaining: 15.6s
331:	learn: 0.5887380	total: 7.73s	remaining: 15.5s
332:	learn: 0.5878436	total: 7.75s	remaining: 15.5s
333:	learn: 0.5875476	total: 7.77s	remaining: 15.5s
334:	learn: 0.5869738	total: 7.79s	remaining: 15.5s
335:	learn: 0.5867249	total: 7.81s	remaining: 15.4s
336:	learn: 0.5858278	total: 7.83s	remaining: 15.4s
337:	learn: 0.5854322	total: 7.84s	remaining: 15.4s
338:	learn: 0.5846967	total: 7.87s	remaining: 15.3s
339:	learn: 0.5843381	total: 7.89s	remaining: 15.3s
340:	learn: 0.5840536	total: 7.92s	remaining: 15.3s
341:	learn: 0.5834569	total: 7.95s	remaining: 15.3s
342:	learn: 0.5827068	total: 7.97s	remaining: 15.3s
343:	learn: 0.5819935	total: 7.99s	remaining: 15.2s
344:	learn: 

484:	learn: 0.5212656	total: 10.7s	remaining: 11.4s
485:	learn: 0.5210101	total: 10.7s	remaining: 11.4s
486:	learn: 0.5205240	total: 10.8s	remaining: 11.3s
487:	learn: 0.5201650	total: 10.8s	remaining: 11.3s
488:	learn: 0.5199706	total: 10.8s	remaining: 11.3s
489:	learn: 0.5196028	total: 10.8s	remaining: 11.3s
490:	learn: 0.5193521	total: 10.8s	remaining: 11.2s
491:	learn: 0.5191847	total: 10.9s	remaining: 11.2s
492:	learn: 0.5187091	total: 10.9s	remaining: 11.2s
493:	learn: 0.5182282	total: 10.9s	remaining: 11.2s
494:	learn: 0.5180392	total: 10.9s	remaining: 11.1s
495:	learn: 0.5175749	total: 10.9s	remaining: 11.1s
496:	learn: 0.5171592	total: 11s	remaining: 11.1s
497:	learn: 0.5168421	total: 11s	remaining: 11.1s
498:	learn: 0.5161996	total: 11s	remaining: 11.1s
499:	learn: 0.5158736	total: 11s	remaining: 11s
500:	learn: 0.5156191	total: 11.1s	remaining: 11s
501:	learn: 0.5153385	total: 11.1s	remaining: 11s
502:	learn: 0.5151091	total: 11.1s	remaining: 11s
503:	learn: 0.5147535	total:

648:	learn: 0.4659789	total: 14.6s	remaining: 7.88s
649:	learn: 0.4656710	total: 14.6s	remaining: 7.87s
650:	learn: 0.4654208	total: 14.6s	remaining: 7.84s
651:	learn: 0.4652408	total: 14.7s	remaining: 7.82s
652:	learn: 0.4648057	total: 14.7s	remaining: 7.8s
653:	learn: 0.4644226	total: 14.7s	remaining: 7.78s
654:	learn: 0.4642379	total: 14.7s	remaining: 7.75s
655:	learn: 0.4636379	total: 14.7s	remaining: 7.73s
656:	learn: 0.4634508	total: 14.8s	remaining: 7.71s
657:	learn: 0.4629993	total: 14.8s	remaining: 7.68s
658:	learn: 0.4628061	total: 14.8s	remaining: 7.66s
659:	learn: 0.4626040	total: 14.8s	remaining: 7.63s
660:	learn: 0.4622272	total: 14.8s	remaining: 7.61s
661:	learn: 0.4618903	total: 14.9s	remaining: 7.59s
662:	learn: 0.4615728	total: 14.9s	remaining: 7.57s
663:	learn: 0.4613569	total: 14.9s	remaining: 7.54s
664:	learn: 0.4610024	total: 14.9s	remaining: 7.52s
665:	learn: 0.4606441	total: 15s	remaining: 7.5s
666:	learn: 0.4603837	total: 15s	remaining: 7.48s
667:	learn: 0.4597

812:	learn: 0.4159603	total: 18.1s	remaining: 4.17s
813:	learn: 0.4155995	total: 18.2s	remaining: 4.15s
814:	learn: 0.4151852	total: 18.2s	remaining: 4.13s
815:	learn: 0.4146381	total: 18.2s	remaining: 4.11s
816:	learn: 0.4145458	total: 18.2s	remaining: 4.08s
817:	learn: 0.4141696	total: 18.3s	remaining: 4.06s
818:	learn: 0.4139692	total: 18.3s	remaining: 4.04s
819:	learn: 0.4136841	total: 18.3s	remaining: 4.01s
820:	learn: 0.4132788	total: 18.3s	remaining: 3.99s
821:	learn: 0.4130190	total: 18.3s	remaining: 3.97s
822:	learn: 0.4128209	total: 18.4s	remaining: 3.95s
823:	learn: 0.4124165	total: 18.4s	remaining: 3.93s
824:	learn: 0.4121026	total: 18.4s	remaining: 3.9s
825:	learn: 0.4116796	total: 18.4s	remaining: 3.88s
826:	learn: 0.4114790	total: 18.4s	remaining: 3.86s
827:	learn: 0.4112893	total: 18.5s	remaining: 3.83s
828:	learn: 0.4111269	total: 18.5s	remaining: 3.81s
829:	learn: 0.4109996	total: 18.5s	remaining: 3.79s
830:	learn: 0.4107465	total: 18.5s	remaining: 3.77s
831:	learn: 0

972:	learn: 0.3750229	total: 21.5s	remaining: 598ms
973:	learn: 0.3747780	total: 21.6s	remaining: 576ms
974:	learn: 0.3745318	total: 21.6s	remaining: 554ms
975:	learn: 0.3743050	total: 21.6s	remaining: 531ms
976:	learn: 0.3739163	total: 21.6s	remaining: 509ms
977:	learn: 0.3735997	total: 21.6s	remaining: 487ms
978:	learn: 0.3733644	total: 21.7s	remaining: 465ms
979:	learn: 0.3731150	total: 21.7s	remaining: 443ms
980:	learn: 0.3726832	total: 21.7s	remaining: 421ms
981:	learn: 0.3725567	total: 21.7s	remaining: 398ms
982:	learn: 0.3722277	total: 21.8s	remaining: 376ms
983:	learn: 0.3719026	total: 21.8s	remaining: 354ms
984:	learn: 0.3717511	total: 21.8s	remaining: 332ms
985:	learn: 0.3713684	total: 21.8s	remaining: 310ms
986:	learn: 0.3709898	total: 21.9s	remaining: 288ms
987:	learn: 0.3705737	total: 21.9s	remaining: 266ms
988:	learn: 0.3703740	total: 21.9s	remaining: 244ms
989:	learn: 0.3699954	total: 21.9s	remaining: 222ms
990:	learn: 0.3698253	total: 22s	remaining: 199ms
991:	learn: 0.

In [424]:
pred_y = model.predict(X_test)

In [425]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.694


# Xgboost

In [413]:
from xgboost import XGBClassifier

In [414]:
# XGB는 생각보다 성능이 낮다... 아마 파라미터 조정을 해야될듯...
model = XGBClassifier(n_estimators = 500)

In [415]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [416]:
score.mean()

0.6842969657085627

In [418]:
score

array([0.68222768, 0.69155045, 0.69047619, 0.66447368, 0.68832237,
       0.70510708, 0.6878089 , 0.66749175, 0.68894389, 0.67656766])

In [79]:
from lightgbm import LGBMClassifier

In [207]:
model = LGBMClassifier(n_estimators = 500)

In [208]:
# non-ascii 문제가 생긴다... 해결 방법 찾아봐야할 듯. 
model.fit(X_train,y_train)

LightGBMError: Do not support non-ASCII characters in feature name.

In [151]:
pred_y = model.predict(X_test)
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.577
